# **UNIVERSIDAD TORCUATO DI TELLA**
## **MAESTRÍA EN ECONOMETRÍA**

---

### **TRABAJO PRÁCTICO DE ECONOMETRÍA**

- **Profesor:** González-Rozada, Martín  
- **Ayudante:** Lening, Iara  
- **Alumno:** Guzzi, David Alexander  (Legajo n°: 24H1970, DNI: 37.703.649)  

**Ciclo Lectivo:** Tercer Trimestre, 2024  

---

#### **1. PROPIEDADES DE MUESTRA FINITA DE FGLS (MCGE).**

Considere el siguiente modelo:

$$
\begin{equation}
y_i = \beta_0 + \beta_1 x_i + u_i, \quad i = 1, 2, \dots, 5N
\tag{1}
\end{equation}
$$

Donde:

- $\beta_0 = -3$
- $\beta_1 = 0.8$
- $x_j \sim U[1, 50]$
- $u_j \sim N(0, \omega \cdot I_{N \times N})$, con la matriz $\Omega$ definida como:

$$
\Omega = \begin{bmatrix}
4 & 0 & 0 & 0 & 0 \\
0 & 9 & 0 & 0 & 0 \\
0 & 0 & 16 & 0 & 0 \\
0 & 0 & 0 & 25 & 0 \\
0 & 0 & 0 & 0 & 36
\end{bmatrix}
$$

In [ ]:
# Importación de librerías.
import numpy as np
import pandas as pd
from scipy.stats import t, chi2

# Semilla para la generación de números aleatorios.
np.random.seed(3649)

# Configuración de pandas (4 decimales).
pd.set_option('display.float_format', '{:.4f}'.format)

**Se presentan tres funciones que se utilizarán en el apartado 1:**
- **fgls_estimation:** Realiza estimación FGLS White;
- **simulate_fgls:** realiza simulaciones utilizando FGLS White;
- **gls_estimation:** realiza estimación y simulaciones para GLS.

**Observación:** Se prescinde del uso de librerías como *statsmodels*, que incluyen implementaciones de OLS y GLS, con el fin de aplicar las ecuaciones vistas en clase, lo que aporta mayor claridad. Para operaciones matriciales, se utiliza la librería *numpy*.

In [ ]:
def fgls_estimation(x: np.ndarray, y: np.ndarray) -> tuple[float, float, float]:
    
    """
    Realiza una estimación de Feasible Generalized Least Squares (FGLS) 
    para un modelo lineal con errores heterocedásticos.

    Parámetros
    ----------
    x : np.ndarray
        Variable regresora.
    y : np.ndarray
        Variable de respuesta.

    Retorna
    -------
    tuple[float, float, float]
        beta0_fgls : float
            Intercepto estimado.
        beta1_fgls : float
            Coeficiente estimado para el regresor.
        se_beta1_fgls : float
            Error estándar del coeficiente estimado para el regresor.
    """

    # 1. Estimar modelo inicial y ~ x por OLS y obtener las estimaciones de los parámetros del modelo.
    X = np.column_stack((np.ones_like(x), x)) # Se construye matriz de regresores, contemplando intercepto.
    beta_ols = np.linalg.inv(X.T @ X) @ X.T @ y # Una alternativa es usar np.linalg.lstsq(X, y, rcond=None)[0] (más eficiente). 

    # 2. Calcular los residuos del modelo, elevarlos al cuadrado y obtener el logaritmo de los mismos.
    u_hat = y - X @ beta_ols
    u_hat2 = u_hat ** 2  
    log_u_hat2 = np.log(u_hat2) #Obs.: Luego de varias iteraciones, se considera que de esta forma se obtienen resultados más coherentes.

    # 3. Dada la forma funcional de la heterocedasticidad de White para este modelo, sigma2 ~ x + x2, 
    # se estima por OLS usando u_hat2 como proxy de sigma2.
    X_aux = np.column_stack((x, x**2))
    gamma_hat = np.linalg.inv(X_aux.T @ X_aux) @ X_aux.T @ log_u_hat2
    
    # 4. Usar las estimaciones de la regresión auxiliar y obtener las varianzas ajustadas (no consistentes).
    log_sigma2_hat = X_aux @ gamma_hat
    sigma2_hat = np.exp(log_sigma2_hat) 

    # 5. Transformar las variables de la forma funcional de la heterocedasticidad de White dividiéndolas por sigma2_hat y estimar por OLS.
    X_aux2 = np.column_stack((x / sigma2_hat,  x**2 / sigma2_hat))
    u_hat2_over_sigma2_hat = u_hat2 / sigma2_hat # Se omite el logaritmo, ya que se considera que no es necesario y así se obtienen resultados más coherentes.
    gamma_hat2 = np.linalg.inv(X_aux2.T @ X_aux2) @ X_aux2.T @ u_hat2_over_sigma2_hat

    # 6. Usar las estimaciones de la segunda regresión auxiliar y obtener las varianzas ajustadas (consistentes).
    sigma2_tilde = np.maximum(X_aux @ gamma_hat2, 1e-6) # Se asegura no negatividad de las varianzas. 
    omega_tilde_inv = np.linalg.inv(np.diag(sigma2_tilde))
    
    # 7. Aplicar GLS en la regresión de y ~ x, utilizando como ponderador la matriz omega estimada.
    # Obs.: También se podría usar uno sobre la raíz cuadrada de sigma2 tilde como ponderador en la regresión OLS de y ~ x.
    
    # Se obtienen las estimaciones de los parámetros.
    beta_fgls = np.linalg.inv(X.T @ omega_tilde_inv @ X) @ X.T @ omega_tilde_inv @ y
    
    # Se obtienen las estimaciones de los errores estándar de los parámetros.
    residuals_fgls = y - X @ beta_fgls
    s2_fgls = (residuals_fgls.T @ omega_tilde_inv @ residuals_fgls) / ( X.shape[0] - X.shape[1])
    var_beta_fgls = s2_fgls * np.linalg.inv(X.T @ omega_tilde_inv @ X)
    se_beta1_fgls = np.sqrt(np.diag(var_beta_fgls))[1]   
    
    return beta_fgls[0], beta_fgls[1], se_beta1_fgls

In [ ]:
def simulate_fgls(n_samples: int, n_observations_list: list[int], beta_0_true: float, beta_1_true: float, beta1_H0: float) -> pd.DataFrame:
    
    """
    Simula la estimación de FGLS para distintos tamaños de muestra, y evalúa la validez de un test de hipótesis.

    Parámetros:
    -----------
    n_samples : int
        Número de simulaciones a realizar para cada tamaño de muestra.
    n_observations_list : list of int
        Lista con los distintos tamaños de muestra a evaluar.
    beta_0_true : float
        Valor verdadero del intercepto en la simulación.
    beta_1_true : float
        Valor verdadero del coeficiente del regresor en la simulación.
    beta1_H0 : float
        Valor hipotético de beta_1 bajo H0 para el test de hipótesis.

    Retorna:
    --------
    results_df : pandas.DataFrame
        DataFrame con estadísticas de los coeficientes estimados y tasas de rechazo del test de hipótesis.
        Columnas:
        - 'n_observations': Tamaño de la muestra.
        - 'mean_beta0': Media de las estimaciones de beta_0.
        - 'median_beta0': Mediana de las estimaciones de beta_0.
        - 'std_beta0': Desviación estándar de las estimaciones de beta_0.
        - 'mean_beta1': Media de las estimaciones de beta_1.
        - 'median_beta1': Mediana de las estimaciones de beta_1.
        - 'std_beta1': Desviación estándar de las estimaciones de beta_1.
        - 'test_size_1pct': Proporción de rechazos de H0 al 1% de significancia.
        - 'test_size_5pct': Proporción de rechazos de H0 al 5% de significancia.
    """

    
    results = []
    # Iteración sobre los tamaños de muestra.
    for n_obs in n_observations_list:
        sample_results = []
        
        # Iteración/simulaciones sobre cada observación de la muestra.
        for _ in range(n_samples):
            # Generación de datos.
            x = np.random.uniform(1, 50, n_obs) # x ~ U[1, 50].
            u = np.random.normal(scale=x) # u ~ N(0, omega), con varianza heterocedástica dependiendo de x.
            y = beta_0_true + beta_1_true * x + u
            
            # Estimación FGLS.
            beta_0_hat, beta_1_hat, se_beta1_hat = fgls_estimation(x, y)
            
            # Test de hipótesis para beta_1 = beta1_H0.
            t_stat = (beta_1_hat - beta1_H0) / se_beta1_hat
            p_value = 2 * (1 - t.cdf(abs(t_stat), df=n_obs - 2))
            reject_1pct = p_value < 0.01 # Rechazar H0 al 1% de significancia.
            reject_5pct = p_value < 0.05 # Rechazar H0 al 5% de significancia.
            
            sample_results.append([beta_0_hat, beta_1_hat, reject_1pct, reject_5pct])
        
        # Convertir a DataFrame.
        df_results = pd.DataFrame(sample_results, columns=['beta_0_hat', 'beta_1_hat', 'reject_1pct', 'reject_5pct'])
        
        # Calcular estadísticas.
        mean_beta0 = df_results['beta_0_hat'].mean()
        mean_beta1 = df_results['beta_1_hat'].mean()
        median_beta0 = df_results['beta_0_hat'].median()
        median_beta1 = df_results['beta_1_hat'].median()
        std_beta0 = df_results['beta_0_hat'].std()
        std_beta1 = df_results['beta_1_hat'].std()
        test_1pct = df_results['reject_1pct'].mean() #Equivale a la proporción de rechazos de las 5000 simulaciones al 1% de significancia.
        test_5pct = df_results['reject_5pct'].mean() #Equivale a la proporción de rechazos de las 5000 simulaciones al 5% de significancia.
        
        results.append([n_obs, mean_beta0, median_beta0, std_beta0, mean_beta1, median_beta1, std_beta1, test_1pct, test_5pct])
    
    results_df = pd.DataFrame(results, columns=['n_observations', 'mean_beta0', 'median_beta0', 'std_beta0', 'mean_beta1', 'median_beta1', 'std_beta1', 'test_1pct', 'test_5pct'])
    
    return results_df

In [ ]:
def simulate_gls(n_samples: int, n_observations: int, beta_0_true: float, beta_1_true: float, beta1_H0: float, omega: np.ndarray) -> pd.DataFrame:
    """
    Simula la estimación de GLS para un determinado tamaño de muestra, y evalúa la validez de un test de hipótesis.

    Parámteros:
    ----------
    n_samples : int
        Número de simulaciones a realizar para cada tamaño de muestra.
    n_observations : int
        Tamaño de la muestra.
    beta_0_true : float
        Valor verdadero del intercepto en la simulación.
    beta_1_true : float
        Valor verdadero del coeficiente del regresor en la simulación.
    beta1_H0 : float
        Valor hipotético de beta_1 bajo H0 para el test de hipótesis.
    omega : np.ndarray
        Matriz de varianzas y covarianzas de los errores.

    Retorna:
    --------
    results_df : pd.DataFrame
        DataFrame con estadísticas de los coeficientes estimados y tasas de rechazo del test de hipótesis.
        Columnas:
        - 'n_observations': Tamaño de la muestra.
        - 'mean_beta0': Media de las estimaciones de beta_0.
        - 'median_beta0': Mediana de las estimaciones de beta_0.
        - 'std_beta0': Desviación estándar de las estimaciones de beta_0.
        - 'mean_beta1': Media de las estimaciones de beta_1.
        - 'median_beta1': Mediana de las estimaciones de beta_1.
        - 'std_beta1': Desviación estándar de las estimaciones de beta_1.
        - 'test_size_1pct': Proporción de rechazos de H0 al 1% de significancia.
        - 'test_size_5pct': Proporción de rechazos de H0 al 5% de significancia.
    """
    
    sample_results = []

    # Descomposición de Cholesky de omega.
    omega = np.diag(omega)
    P = np.linalg.cholesky(omega)
    P_inv = np.linalg.inv(P)

    # Iteración/simulaciones sobre cada observación de la muestra.
    for _ in range(n_samples):
        # Generación de datos.
        x = np.random.uniform(1, 50, n_observations)  # x ~ U[1, 50]
        u = np.random.multivariate_normal(mean=np.zeros(n_observations), cov=omega)  # u ~ N(0, omega)
        y = beta_0_true + beta_1_true * x + u

        # Transformación para GLS.
        y_tilde = P_inv @ y
        X = np.column_stack((np.ones(n_observations), x)) # Se construye matriz de regresores, contemplando intercepto.
        X_tilde = P_inv @ X

        # Ajuste modelo transformado con OLS para obtener estimadores y desvío estándar de los mismos.
        beta_hat = np.linalg.inv(X_tilde.T @ X_tilde) @ (X_tilde.T @ y_tilde)
        residuals = y_tilde - X_tilde @ beta_hat
        s2 = (residuals @ residuals) / (n_observations - X_tilde.shape[1])
        var_beta_hat = s2 * np.linalg.inv(X_tilde.T @ X_tilde)
        se_beta1_hat = np.sqrt(np.diag(var_beta_hat))[1]

        # Test de hipótesis para beta_1 = beta1_H0 (tamaño del test).
        t_stat = (beta_hat[1] - beta1_H0) / se_beta1_hat
        p_value = 2 * (1 - t.cdf(abs(t_stat), df=n_observations - 2))
        reject_1pct = p_value < 0.01  # Rechazar H0 al 1% de significancia.
        reject_5pct = p_value < 0.05  # Rechazar H0 al 5% de significancia.

        sample_results.append([beta_hat[0], beta_hat[1], reject_1pct, reject_5pct])

    # Convertir a DataFrame.
    df_results = pd.DataFrame(sample_results, columns=['beta_0_hat', 'beta_1_hat', 'reject_1pct', 'reject_5pct'])

    # Calcular estadísticas.
    results = {
        'n_observations': n_observations,
        'mean_beta0': df_results['beta_0_hat'].mean(),
        'median_beta0': df_results['beta_0_hat'].median(),
        'std_beta0': df_results['beta_0_hat'].std(),
        'mean_beta1': df_results['beta_1_hat'].mean(),
        'median_beta1': df_results['beta_1_hat'].median(),
        'std_beta1': df_results['beta_1_hat'].std(),
        'test_size_1pct': df_results['reject_1pct'].mean(),
        'test_size_5pct': df_results['reject_5pct'].mean()
    }

    return pd.DataFrame([results])

**Ejercicios.**
1. Genere **5000 muestras de 5N = 5 observaciones** de corte transversal a partir del modelo (1):
     
   **a)** Para cada muestra estime por **FGLS** los parámetros del modelo y realice un test de hipótesis para contrastar que $H_0$ : $\beta_1$  = 0.8. Reporte **tamaño del test** al 1% y 5% y el **poder del test** cuando $\beta_1$ = 0 y $\beta_1$ = 0.4. Adicionalmente, reporte la media, mediana y desvio estándar de las estimaciones de $\beta_0$ y $\beta_1$.

   **b)** Utilice la **descomposición de Cholesky** para encontrar una matriz $\Omega = P P'$ y aplique la transformación correspondiente a **GLS** al modelo (1) y estime el modelo por **OLS**, comente si cambia algún resultado.

2. Repita el punto anterior (sólo el inciso a)) con **5N = 10**.

3. Repita el punto anterior (sólo el inciso a)) con **5N = 30**.

4. Repita el punto anterior (sólo el inciso a)) con **5N = 100**.

5. Repita el punto anterior (sólo el inciso a)) con **5N = 200**.

6. Repita el punto anterior (sólo el inciso a)) con **5N = 500**.

7. Describa detalladamente las **propiedades de muestra finita de FGLS** de acuerdo a lo que observó de los cuatro puntos anteriores. En especial, explique **cómo cambia el tamaño y el poder de los tests** a medida que aumenta el tamaño de muestra.

**Ejercicios 1a, 2, 3, 4, 5, 6.**

In [ ]:
# Se generan 5.000 muestras para 5N = 5, 10, 30, 100, 200, 500. Se fija beta_0 = -3 y beta_1 = 0.8, y se obtienen estadísticas descriptivas. Se evalúa H0: beta_1 = 0.8 (tamaño del test). 
simulate_fgls(n_samples=5000, n_observations_list=[5, 10, 30, 100, 200, 500], beta_0_true=-3, beta_1_true=0.8, beta1_H0=0.8)


In [ ]:
# Adicionalmente, se generan 5.000 muestras para 5N = 700, 1000, 1200, 1500, 2000 para observar en qué tamaño de muestra se obtiene un tamaño de test más cercano al nivel de significancia.
simulate_fgls(n_samples=5000, n_observations_list=[700, 1000, 1200, 1500, 2000], beta_0_true=-3, beta_1_true=0.8, beta1_H0=0.8)


,n_observations,mean_beta0,median_beta0,std_beta0,mean_beta1,median_beta1,std_beta1,test_1pct,test_5pct
0,700,-3.0200,-3.0115,1.8553,0.8116,0.7993,1.6837,0.0292,0.0690
1,1000,-3.0054,-2.9938,1.6094,0.8088,0.8000,1.4567,0.0236,0.0606
2,1200,-2.9859,-2.9945,1.1024,0.7919,0.8000,0.9931,0.0166,0.0584
3,1500,-2.9848,-3.0003,0.8170,0.7858,0.8009,0.7410,0.0146,0.0528
4,2000,-2.9913,-2.9926,0.2693,0.7976,0.7992,0.1837,0.0112,0.0472


In [ ]:
# Se generan 5.000 muestras para 5N = 5, 10, 30, 100, 200, 500. Se fija beta_0 = -3 y beta_1 = 0.8, y se obtienen estadísticas descriptivas. Se evalúa H0: beta_1 = 0.4 (poder del test). 
simulate_fgls(n_samples=5000, n_observations_list=[5, 10, 30, 100, 200, 500], beta_0_true=-3, beta_1_true=0.8, beta1_H0=0.4)

,n_observations,mean_beta0,median_beta0,std_beta0,mean_beta1,median_beta1,std_beta1,test_1pct,test_5pct
0,5,9.2582,-2.6233,893.4464,0.4592,0.7869,22.2380,0.1182,0.2262
1,10,18.4817,-2.9402,994.5864,0.3145,0.7957,23.2917,0.1344,0.2470
2,30,-14.9002,-2.9858,630.7914,1.0043,0.8017,13.5219,0.2010,0.3766
3,100,-4.3804,-3.0011,88.5079,0.8393,0.8005,3.1786,0.6884,0.8616
4,200,-2.9602,-2.9970,8.5433,0.8055,0.8000,2.8978,0.9600,0.9864
5,500,-2.9962,-2.9792,2.0976,0.8099,0.7984,1.7883,0.9964,0.9974


In [ ]:
# Se generan 5.000 muestras para 5N = 5, 10, 30, 100, 200, 500. Se fija beta_0 = -3 y beta_1 = 0.8, y se obtienen estadísticas descriptivas. Se evalúa H0: beta_1 = 0 (poder del test). 
simulate_fgls(n_samples=5000, n_observations_list=[5, 10, 30, 100, 200, 500], beta_0_true=-3, beta_1_true=0.8, beta1_H0=0)

,n_observations,mean_beta0,median_beta0,std_beta0,mean_beta1,median_beta1,std_beta1,test_1pct,test_5pct
0,5,1.7059,-3.0860,1245.7746,0.5090,0.7938,33.4404,0.1472,0.2760
1,10,75.3163,-2.9185,4386.8609,-0.9077,0.7989,98.9208,0.2142,0.3862
2,30,-11.9590,-3.0124,559.0582,0.9417,0.7998,11.6853,0.6186,0.7944
3,100,-3.3895,-2.9896,35.6139,0.7892,0.7953,3.2351,0.9848,0.9896
4,200,-3.0301,-2.9915,3.4799,0.8054,0.7994,2.1781,0.9950,0.9962
5,500,-3.0554,-3.0077,2.0244,0.8435,0.7993,1.7780,0.9968,0.9974


**Ejercicio 1b.**

In [38]:
simulate_gls(n_samples=5000, n_observations=5, beta_0_true=-3, beta_1_true=0.8, beta1_H0=0.8, omega=[4, 9, 16, 25, 36])

,n_observations,mean_beta0,median_beta0,std_beta0,mean_beta1,median_beta1,std_beta1,test_size_1pct,test_size_5pct
0,5,-2.9583,-2.8909,4.6492,0.7981,0.7964,0.1620,0.0068,0.0460


#### **2. PROPIEDADES DE MUESTRA FINITA DEL TEST DE WHITE Y DE LA CORRECIÓN POR HETEROCEDASTICIDAD.**

Consideremos el siguiente modelo:

$$
y_i = \beta_0 + \beta_1 x_{1i} + \beta_2 x_{2i} + \sqrt{v_i} u_i, \quad i = 1, \dots, n \tag{2}
$$

Para $n = 20$, $x_1$ se determina como una secuencia de 18 puntos espaciados uniformemente entre -1 y 1 con puntos extremos dados por -1.1 y 1.1, mientras que  $x_2$ son cuantiles de una distribución normal estándar elegidos aleatoriamente. La generación de los datos de la variable dependiente se hace con  $\beta_0 = \beta_1  = \beta_2$ = 1.

Existen tres diseños:

1. **Diseño 0**: $u_i \sim N(0,1)$, y $v_i = 1$ (Normalidad y homocedasticidad);

2. **Diseño 1**: $u_i \sim N(0,1)$, y $v_i = e^{0.25 x_{1i} + 0.25 x_{2i}}$ (Normalidad y heterocedasticidad);

3. **Diseño 2**: $u_i \sim t_5$, y $v_i = e^{0.25 x_{1i} + 0.25 x_{2i}}$ (No-normalidad y heterocedasticidad).

Todas las simulaciones se basan en **5.000 replicaciones**.




##### **2.1. TEST DE HETEROCEDASTICIDAD DE WHITE.**

**Ejercicios.**

**a)** Para el **diseño 0**, genere 5.000 muestras de $n = 20$ observaciones a partir del modelo (2). Para cada muestra, **estime por OLS** los parámetros del modelo y realice el **test de hipótesis de White** para contrastar que $H_0$ : No hay heterocedasticidad. Reporte el **tamaño del test** al 1%, 5%, 10%.

**b)** Para los **diseños 1 y 2**, genere 5.000 muestras de $n = 20$ observaciones a partir del modelo (2) y reporte el **poder del test cuando el diseño utilizado es el modelo poblacional verdadero**.

**c)** Repita el punto anterior con $n = 60$.

**d)** Repita el punto anterior con $n = 100$.

**e)** Repita el punto anterior con $n = 200$.

**f)** Repita el punto anterior con $n = 400$.

**g)** Repita el punto anterior con $n = 600$.

**h)** Describa detalladamente las **propiedades de muestra finita del test de White** de acuerdo a lo observado en los puntos anteriores.


In [46]:
def white_het_test(n_samples: int, n_observations_list: list[int], beta_0_true: float, beta_1_true: float, beta_2_true: float, design: int) -> pd.DataFrame:
    
    
    results = []
    for n_obs in n_observations_list:
        
        # Resultados para almacenar p-values.
        test_results = []

        for _ in range(n_samples):

            # Generación de datos
            x1_core = np.linspace(-1, 1, n_obs-2)  # n_observations-2 puntos uniformes entre -1 y 1.
            x1 = np.concatenate([[-1.1], x1_core, [1.1]])  # Agregar extremos -1.1 y 1.1.
            x2 = np.random.normal(0, 1, n_obs)
            X = np.column_stack((np.ones_like(x1), x1, x2)) #Se agrega intercepto.

            if design == 0:
                u = np.random.normal(0, 1, n_obs) # u normal
                v = np.ones(n_obs)  # v = 1
            elif design == 1:
                u = np.random.normal(0, 1, n_obs) # u normal
                v = np.exp(0.25 * x1 + 0.25 * x2) #v: v = exp(0.25*x1 + 0.25*x2)
            elif design == 2:
                u = np.random.standard_t(5, n_obs)# u t-student
                v = np.exp(0.25 * x1 + 0.25 * x2) #v: v = exp(0.25*x1 + 0.25*x2)
            else:
                raise ValueError("Invalid design parameter. Choose 0, 1 or 2.")

            y = beta_0_true + beta_1_true * x1 + beta_2_true * x2 + np.sqrt(v) * u

            # OLS Estimación
            beta_ols = np.linalg.inv(X.T @ X) @ X.T @ y
            residuals_ols = y - X @ beta_ols
            residuals2_ols = residuals_ols ** 2

            # Variables auxiliares
            X_aux_wc = np.column_stack((x1, x2, x1 ** 2, x2 ** 2, x1 * x2))
            X_aux = np.column_stack((X, x1 ** 2, x2 ** 2, x1 * x2))

            # Estimación del modelo auxiliar
            beta_ols_aux = np.linalg.inv(X_aux.T @ X_aux) @ X_aux.T @ residuals2_ols

            # Cálculo de R^2
            residuals_ols_aux = residuals2_ols - X_aux @ beta_ols_aux
            ss_total = np.sum((residuals2_ols - np.mean(residuals2_ols)) ** 2)
            ss_residual = np.sum(residuals_ols_aux ** 2)
            r_squared = 1 - (ss_residual / ss_total)

            # Estadístico LM
            lm_stat = n_obs * r_squared

            # P-valor de LM
            p_value = chi2.sf(lm_stat, X_aux_wc.shape[1])

            reject_1pct = p_value < 0.01 # Rechazar H0 al 1% de significancia.
            reject_5pct = p_value < 0.05 # Rechazar H0 al 5% de significancia.
            reject_10pct = p_value < 0.10 # Rechazar H0 al 10% de significancia.

            test_results.append([reject_1pct, reject_5pct, reject_10pct])

        # Convertir los resultados a un DataFrame
        test_results_df = pd.DataFrame(test_results, columns=['reject_1pct', 'reject_5pct', 'reject_10pct'])
        
        # Calcular estadísticas.
        test_1pct = test_results_df['reject_1pct'].mean() #Equivale a la proporción de rechazos de las 5000 simulaciones al 1% de significancia.
        test_5pct = test_results_df['reject_5pct'].mean() #Equivale a la proporción de rechazos de las 5000 simulaciones al 5% de significancia.
        test_10pct = test_results_df['reject_10pct'].mean() #Equivale a la proporción de rechazos de las 5000 simulaciones al 10% de significancia.
        
        results.append([n_obs, test_1pct, test_5pct, test_10pct])

    results_df = pd.DataFrame(results, columns=['n_observations', 'test_1pct', 'test_5pct', 'test_10pct'])

    return results_df

In [ ]:
# Se generan 5.000 muestras para n = 20. Se fija beta_0 = beta_1 = beta_1 = 1, y diseño de errores normales y homocedásticos. 
# Se evalúa H0: No hay heterocedasticidad (tamaño del test).
white_het_test(n_samples=5000, n_observations_list=[20], beta_0_true=1, beta_1_true=1, beta_2_true=1, design=0)

,n_observations,test_1pct,test_5pct,test_10pct
0,20,0.0016,0.0376,0.0898


In [50]:
# Adicionalmente, se generan 5.000 muestras para n = 60, 100, 200, 400, 600 para comprobar que el tamaño del test se estabiliza en torno al nivel de significancia.
white_het_test(n_samples=5000, n_observations_list=[60, 100, 200, 400, 600], beta_0_true=1, beta_1_true=1, beta_2_true=1, design=0)

,n_observations,test_1pct,test_5pct,test_10pct
0,60,0.0114,0.0514,0.0950
1,100,0.0110,0.0508,0.0908
2,200,0.0106,0.0448,0.0942
3,400,0.0132,0.0542,0.0994
4,600,0.0118,0.0534,0.1004


In [ ]:
# Se generan 5.000 muestras para n = 20, 60, 100, 200, 400, 600. Se fija beta_0 = beta_1 = beta_1 = 1, y diseño de errores normales y heterocedásticos. 
# Se evalúa H0: No hay heterocedasticidad (poder del test).
white_het_test(n_samples=5000, n_observations_list=[20, 60, 100, 200, 400, 600], beta_0_true=1, beta_1_true=1, beta_2_true=1, design=1)

,n_observations,test_1pct,test_5pct,test_10pct
0,20,0.0072,0.0586,0.1196
1,60,0.0580,0.1424,0.2274
2,100,0.1048,0.2396,0.3434
3,200,0.2768,0.4986,0.6282
4,400,0.6634,0.8504,0.9160
5,600,0.9026,0.9710,0.9854


In [49]:
# Se generan 5.000 muestras para n = 20, 60, 100, 200, 400, 600. Se fija beta_0 = beta_1 = beta_1 = 1, y diseño de errores no-normales y heterocedásticos. 
# Se evalúa H0: No hay heterocedasticidad (poder del test).
white_het_test(n_samples=5000, n_observations_list=[20, 60, 100, 200, 400, 600], beta_0_true=1, beta_1_true=1, beta_2_true=1, design=2)

,n_observations,test_1pct,test_5pct,test_10pct
0,20,0.0088,0.0698,0.1310
1,60,0.0518,0.1222,0.1830
2,100,0.0712,0.1592,0.2258
3,200,0.1236,0.2516,0.3518
4,400,0.2596,0.4438,0.5576
5,600,0.3912,0.5946,0.7032


##### **2.2. CORRECIÓN DE LA MATRIZ DE VARIANZAS Y COVARIANZAS EN PRESENCIA DE HETEROCEDASTICIDAD.**

In [10]:
def white_vcov_correction(n_samples: int, n_observations_list: list[int], beta_0_true: float, beta_1_true: float, beta_2_true: float, design: int) -> pd.DataFrame:
    
    
    results = []
    for n_obs in n_observations_list:
        
        # Resultados para almacenar p-values.
        bias_results = []

        for _ in range(n_samples):

            # Generación de datos
            x1_core = np.linspace(-1, 1, n_obs-2)  # n_observations-2 puntos uniformes entre -1 y 1.
            x1 = np.concatenate([[-1.1], x1_core, [1.1]])  # Agregar extremos -1.1 y 1.1.
            x2 = np.random.normal(0, 1, n_obs)
            X = np.column_stack((np.ones_like(x1), x1, x2)) #Se agrega intercepto.

            if design == 1:
                u = np.random.normal(0, 1, n_obs) # u ~ N(0, 1).
                var_u_t = 1  # Varianza teórica.
            elif design == 2:
                u = np.random.standard_t(5, n_obs)# u ~ t-student(5).
                var_u_t = 5 / 3 # Varianza teórica: (var(u) = df / (df - 2)); df: degree of freedom.
            else:
                raise ValueError("Invalid design parameter. Choose 1 or 2.")

            v = np.exp(0.25 * x1 + 0.25 * x2) #v: v = exp(0.25*x1 + 0.25*x2)
            y = beta_0_true + beta_1_true * x1 + beta_2_true * x2 + np.sqrt(v) * u

            # OLS Estimación
            beta_ols = np.linalg.inv(X.T @ X) @ X.T @ y
            residuals_ols = y - X @ beta_ols
            residuals2_ols = residuals_ols ** 2

            # Matriz de varianzas y covarianzas de los errores propuesta por White.
            omega_white = np.diag(residuals2_ols)
            # Matriz de varianzas y covarianzas de los errores poblacional.
            omega_pop = np.diag(v*var_u_t)
            # Matriz de varianzas y covarianzas de los errores propuesta por White, reemplazando residuos por errores.
            omega_white_pop = np.diag(np.power(np.sqrt(v)*u, 2))

            #Matriz de varianzas y covarianzas de los parámetros, para cada matriz de varianzas y covarianzas de los errores.
            vcov_beta_white = np.linalg.inv(X.T @ X) @ X.T @ omega_white @ X @ np.linalg.inv(X.T @ X)
            vcov_beta_pop = np.linalg.inv(X.T @ X) @ X.T @ omega_pop @ X @ np.linalg.inv(X.T @ X)
            vcov_beta_white_pop = np.linalg.inv(X.T @ X) @ X.T @ omega_white_pop @ X @ np.linalg.inv(X.T @ X)

            # Varianzas de los parámetros.
            var_beta0_white, var_beta1_white, var_beta2_white = np.diag(vcov_beta_white)
            var_beta0_pop, var_beta1_pop, var_beta2_pop = np.diag(vcov_beta_pop)
            var_beta0_white_pop, var_beta1_white_pop, var_beta2_white_pop = np.diag(vcov_beta_white_pop)

            # Sesgo relativo y sesgos relativos totales de la estimación de las varianzas de los parámetros (White con residuos).
            bias_var_beta0 = (var_beta0_white - var_beta0_pop) / var_beta0_pop
            bias_var_beta1 = (var_beta1_white - var_beta1_pop) / var_beta1_pop
            bias_var_beta2 = (var_beta2_white - var_beta2_pop) / var_beta2_pop

            # Sesgo relativo y sesgos relativos totales de la estimación de las varianzas de los parámetros (White con errores).
            bias_var_beta0_pop = (var_beta0_white_pop - var_beta0_pop) / var_beta0_pop
            bias_var_beta1_pop = (var_beta1_white_pop - var_beta1_pop) / var_beta1_pop
            bias_var_beta2_pop = (var_beta2_white_pop - var_beta2_pop) / var_beta2_pop

            bias_results.append([bias_var_beta0, bias_var_beta1, bias_var_beta2, bias_var_beta0_pop, bias_var_beta1_pop, bias_var_beta2_pop])
        
        # Convertir los resultados a un DataFrame
        bias_results_df = pd.DataFrame(bias_results, columns=['bias_var_beta0', 'bias_var_beta1', 'bias_var_beta2', 'bias_var_beta0_pop', 'bias_var_beta1_pop', 'bias_var_beta2_pop'])
        
        # Calcular estadísticas.
        bias_var_beta0 = bias_results_df['bias_var_beta0'].mean() 
        bias_var_beta1 = bias_results_df['bias_var_beta1'].mean()
        bias_var_beta2 = bias_results_df['bias_var_beta2'].mean()
        bias_var_total = bias_var_beta0 + bias_var_beta1 + bias_var_beta2
        bias_var_beta0_pop = bias_results_df['bias_var_beta0_pop'].mean()
        bias_var_beta1_pop = bias_results_df['bias_var_beta1_pop'].mean()
        bias_var_beta2_pop = bias_results_df['bias_var_beta2_pop'].mean()
        bias_var_pop_total = bias_var_beta0_pop + bias_var_beta1_pop + bias_var_beta2_pop
        
        results.append([n_obs, bias_var_beta0, bias_var_beta1, bias_var_beta2, bias_var_total, bias_var_beta0_pop, bias_var_beta1_pop, bias_var_beta2_pop, bias_var_pop_total])

    results_df = pd.DataFrame(results, columns=['n_observations', 'bias_var_beta0', 'bias_var_beta1', 'bias_var_beta2', 'bias_var_total', 'bias_var_beta0_pop', 'bias_var_beta1_pop', 'bias_var_pop_total', 'bias_var_total_pop'])
    
    # Formato de columnas.
    cols_ex = ['n_observations']
    cols_in = [col for col in results_df.columns if col not in cols_ex]

    return results_df.style.format({col: '{:.2%}' for col in cols_in})

In [11]:
white_vcov_correction(n_samples=5000, n_observations_list=[20, 60, 100, 200, 400, 600], beta_0_true=1, beta_1_true=1, beta_2_true=1, design=1)

,n_observations,bias_var_beta0,bias_var_beta1,bias_var_beta2,bias_var_total,bias_var_beta0_pop,bias_var_beta1_pop,bias_var_pop_total,bias_var_total_pop
0,20,-16.07%,-19.18%,-24.40%,-59.65%,-0.60%,-0.31%,-0.91%,-1.82%
1,60,-5.10%,-6.47%,-9.19%,-20.77%,0.00%,-0.01%,-0.48%,-0.49%
2,100,-3.03%,-3.73%,-4.99%,-11.75%,0.05%,0.13%,0.37%,0.55%
3,200,-1.42%,-1.68%,-2.48%,-5.58%,0.12%,0.29%,0.30%,0.71%
4,400,-0.78%,-0.88%,-1.28%,-2.94%,-0.01%,0.08%,0.09%,0.16%
5,600,-0.55%,-0.79%,-0.94%,-2.28%,-0.04%,-0.14%,-0.03%,-0.21%


In [12]:
white_vcov_correction(n_samples=5000, n_observations_list=[20, 60, 100, 200, 400, 600], beta_0_true=1, beta_1_true=1, beta_2_true=1, design=2)

,n_observations,bias_var_beta0,bias_var_beta1,bias_var_beta2,bias_var_total,bias_var_beta0_pop,bias_var_beta1_pop,bias_var_pop_total,bias_var_total_pop
0,20,-15.33%,-18.40%,-24.48%,-58.21%,0.75%,0.79%,-0.23%,1.31%
1,60,-5.58%,-5.97%,-8.62%,-20.17%,-0.46%,0.61%,-0.07%,0.08%
2,100,-3.25%,-3.66%,-4.70%,-11.61%,-0.23%,0.19%,0.75%,0.70%
3,200,-1.83%,-2.27%,-2.88%,-6.99%,-0.31%,-0.34%,-0.21%,-0.86%
4,400,-0.75%,-0.70%,-1.48%,-2.93%,0.02%,0.29%,-0.10%,0.21%
5,600,-0.31%,-0.68%,-0.65%,-1.64%,0.21%,-0.01%,0.25%,0.44%
